In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import sys
import json
import urllib.request
import requests
import pandas as pd
import ast
import requests
from datetime import timedelta
import itertools
import numpy as np

In [ ]:
cd /content/drive/Othercomputers/내 노트북/2024-1/데이터마이닝/final project

## **행사정보 데이터프레임 만들기**

In [ ]:
df = pd.read_csv("행사정보_노션2.csv")
df.head()

In [ ]:
event_df = pd.DataFrame()
event_df['start_date'] = df['날짜'].apply(lambda x : x.split(' → ')[0])
event_df['end_date'] = df['날짜'].apply(lambda x : x.split(' → ')[1] if len(x.split(' → ')) > 1 else x.split(' → ')[0])
event_df['hangang'] = df['공원']
event_df['event_name'] = df['공연명']
event_df['keywords'] = df['키워드']

In [ ]:
event_df['start_date'] = pd.to_datetime(event_df['start_date'], format='%Y년 %m월 %d일')
event_df['end_date'] = pd.to_datetime(event_df['end_date'], format='%Y년 %m월 %d일')

In [ ]:
event_df.head()

## **네이버 api로 일별버즈량 가져오기**

In [ ]:
event_dict = {}

for i in range(event_df.shape[0]):
    event = {}
    event['groupName'] = event_df['event_name'][i]
    event['keywords'] = event_df['keywords'][i].split(', ')
    event_dict[i] = event

In [ ]:
groundKey = 26
event_dict[groundKey]

In [ ]:
buzz_raw = []

In [ ]:
client_id = "2GxtSoCwekuwBdWnSU6q"
client_secret = "jsOfHpQ742"
for i in range(0, len(event_dict), 4):
    url = "https://openapi.naver.com/v1/datalab/search";
    testList = []
    if i == 80:
       testList.append(event_dict[i])
    else:
       for j in range(i,i+4):
           testList.append(event_dict[j])
    testList.append(event_dict[groundKey])
    params = {"startDate": "2020-01-01",
              "endDate": "2023-05-31",
              "timeUnit": "date",
              "keywordGroups": testList}
    headers = {"X-Naver-Client-Id": client_id,
               "X-Naver-Client-Secret": client_secret,
               "Content-Type": "application/json"}
    response = requests.post(url, data=json.dumps(params), headers=headers)
    response_data = response.json()
    buzz_raw.append(response_data)

In [ ]:
buzz_list = []

for i in range(len(buzz_raw)):
  for j in range(len(buzz_raw[i]['results'])):
    buzz_list.append(buzz_raw[i]['results'][j])

In [ ]:
buzz_df = pd.DataFrame()

for i in range(len(buzz_list)):
  buzz = pd.DataFrame.from_dict(buzz_list[i]['data'])
  buzz['event_name'] = buzz_list[i]['title']
  buzz_df = pd.concat([buzz_df, buzz])

In [ ]:
buzz_df = buzz_df.drop_duplicates(['period', 'event_name'])

In [ ]:
buzz_df.to_csv('일별버즈량2.csv', index=False, sep=',', encoding='cp949')

## **행사별 버즈량 정보**

### **일별버즈량 데이터 확인 및 전처리**

In [ ]:
buzz_df = pd.read_csv("일별버즈량2.csv", encoding='cp949')
buzz_df.head()

In [ ]:
buzz_df['period'] = pd.to_datetime(buzz_df['period'], format='%Y-%m-%d')
buzz_df.info()

In [ ]:
buzz_df[buzz_df['ratio']==100]

In [ ]:
event_df['event_name'][~event_df['event_name'].isin(buzz_df['event_name'].unique())]

In [ ]:
buzz_df['event_name'].value_counts()

In [ ]:
period = pd.date_range(start='2020-01-01', end='2023-05-31')
event_name = event_df['event_name']
buzz_base = pd.DataFrame(list(itertools.product(event_name, period)), columns=['event_name', 'period'])
buzz_base

In [ ]:
buzz_df = buzz_df.merge(buzz_base, how='right', on=['period', 'event_name'])
buzz_df

In [ ]:
buzz_df.isna().sum()

In [ ]:
buzz_df = buzz_df.fillna(0)
buzz_df.isna().sum()

### **행사별 버즈량 데이터 만들기**

In [ ]:
event_df['before30_date'] = event_df['start_date'] - timedelta(days=30)
event_df['before1_date'] = event_df['start_date'] - timedelta(days=1)
mask = event_df['end_date'] > pd.to_datetime('2023-05-31')
event_df.loc[mask, 'end_date'] = pd.to_datetime('2023-05-31')
event_df.tail()

In [ ]:
merged_df = buzz_df.merge(event_df, how='left', on='event_name')
filtered_df = merged_df[(merged_df['period'] >= merged_df['before30_date']) & (merged_df['period'] <= merged_df['before1_date'])]
filtered_df.head()

In [ ]:
event_df['buzz_sum'] = event_df['event_name'].map(filtered_df.groupby('event_name')['ratio'].sum())
event_df['buzz_max'] = event_df['event_name'].map(filtered_df.groupby('event_name')['ratio'].max())

In [ ]:
event_df.head()

## **데이터 병합**

### **date 생성 및 쉬는날 처리**

In [ ]:
def generate_date_range(start, end):
    start_date = pd.to_datetime(start)
    end_date = pd.to_datetime(end)
    return pd.date_range(start_date, end_date).strftime('%Y-%m-%d').tolist()

expanded_rows = []

for _, row in event_df.iterrows():
    dates = generate_date_range(row['start_date'], row['end_date'])
    hangang_items = row['hangang'].split(', ')

    for date in dates:
        for item in hangang_items:
            new_row = row.copy()
            new_row['date'] = date
            new_row['hangang'] = item
            expanded_rows.append(new_row)

expanded_df = pd.DataFrame(expanded_rows)
expanded_df['date'] = pd.to_datetime(expanded_df['date'], format='%Y-%m-%d')

In [ ]:
expanded_df['weekday'] = expanded_df['date'].dt.weekday

In [ ]:
expanded_df.head()

In [ ]:
date_list = pd.date_range(start='2022-09-09', end='2022-09-10')
condition = (expanded_df['event_name'].isin(['2022 한강달빛야시장'])) &
 ((~expanded_df['weekday'].isin([4, 5])) | (expanded_df['date'].isin(date_list)))
expanded_df.loc[condition, ['buzz_sum', 'buzz_max']] = np.nan

In [ ]:
date_list = pd.date_range(start='2022-09-09', end='2022-09-12')
condition = (expanded_df['event_name'].isin(['2022 잠수교 뚜벅뚜벅 축제'])) &
 ((~expanded_df['weekday'].isin([6])) | (expanded_df['date'].isin(date_list)))
expanded_df.loc[condition, ['buzz_sum', 'buzz_max']] = np.nan

In [ ]:
date_list = pd.date_range(start='2022-09-09', end='2022-09-12')
condition = (expanded_df['event_name'].isin(['2022 한강 이야기 여행 - 한강역사탐방'])) & (expanded_df['date'].isin(date_list))
expanded_df.loc[condition, ['buzz_sum', 'buzz_max']] = np.nan

In [ ]:
date_list = pd.date_range(start='2022-09-09', end='2022-09-12')
condition = (expanded_df['event_name'].isin(['2022 한강 이야기 여행 - 한강야경투어'])) &
 ((~expanded_df['weekday'].isin([3, 4, 5])) | (expanded_df['date'].isin(date_list)))
expanded_df.loc[condition, ['buzz_sum', 'buzz_max']] = np.nan

In [ ]:
date_range_1 = pd.date_range(start='2022-10-07', end='2022-10-09')
date_range_2 = pd.date_range(start='2022-10-14', end='2022-10-16')

date_list = date_range_1.union(date_range_2)

condition = (expanded_df['event_name'].isin(['나홀로 요가'])) & (~expanded_df['date'].isin(date_list))
expanded_df.loc[condition, ['buzz_sum', 'buzz_max']] = np.nan

In [ ]:
date_list = ['2022-10-07','2022-10-14']
date_list = pd.to_datetime(date_list, format='%Y-%m-%d')

condition = (expanded_df['event_name'].isin(['아이언맨 오영환의 자전거교실(유료)'])) & (~expanded_df['date'].isin(date_list))
expanded_df.loc[condition, ['buzz_sum', 'buzz_max']] = np.nan

In [ ]:
date_list = ['2022-10-02','2022-10-09', '2022-10-16']
date_list = pd.to_datetime(date_list, format='%Y-%m-%d')

condition = (expanded_df['event_name'].isin(['한강코딩드론교실'])) & (~expanded_df['date'].isin(date_list))
expanded_df.loc[condition, ['buzz_sum', 'buzz_max']] = np.nan

In [ ]:
condition = (expanded_df['event_name'].isin(['뚝섬 자벌레문화프로그램'])) & (expanded_df['weekday'].isin([0]))
expanded_df.loc[condition, ['buzz_sum', 'buzz_max']] = np.nan

In [ ]:
date_list = ['2023-04-29','2023-05-01', '2023-05-06', '2023-05-07']
date_list = pd.to_datetime(date_list, format='%Y-%m-%d')

condition = (expanded_df['event_name'].isin(['2023 한강불빛공연 드론라이트쇼 (상반기)'])) & (~expanded_df['date'].isin(date_list))
expanded_df.loc[condition, ['buzz_sum', 'buzz_max']] = np.nan

In [ ]:
condition = (expanded_df['event_name'].isin(['2023 한강달빛야시장 상반기 행사'])) & (~expanded_df['weekday'].isin([6]))
expanded_df.loc[condition, ['buzz_sum', 'buzz_max']] = np.nan

In [ ]:
condition = (expanded_df['event_name'].isin(['2023 차 없는 잠수교 뚜벅뚜벅 축제 (상반기)'])) & (~expanded_df['weekday'].isin([6]))
expanded_df.loc[condition, ['buzz_sum', 'buzz_max']] = np.nan

### **데이터 병합**

In [ ]:
expanded_df.isna().sum()

In [ ]:
event_cnt = expanded_df.groupby(['date', 'hangang'])['buzz_sum'].count()
event_cnt = event_cnt.rename('event_cnt').reset_index()

In [ ]:
# 같은 날, 같은 공원에 행사가 여러 개인 경우에 대한 집계 방식은 이 부분을 바꾸면 됨. 현재는 sum() -> 총합으로 들어가 있는 상태
buzz_final = expanded_df.groupby(['date', 'hangang'])[['buzz_sum', 'buzz_max']].sum().reset_index()
buzz_final = buzz_final.merge(event_cnt)
buzz_final

In [ ]:
base = pd.read_csv('hangang+people.csv')
base['date'] = pd.to_datetime(base['date'], format='%Y-%m-%d')
base.head()

In [ ]:
buzz_final = base.merge(buzz_final, how='left', on=['hangang', 'date'])
buzz_final

In [ ]:
13717-1931

In [ ]:
buzz_final.isna().sum()

In [ ]:
buzz_final = buzz_final.fillna(0)
buzz_final.isna().sum()

In [ ]:
buzz_final.to_csv("hangang+people+buzz.csv", index=False, encoding='cp949')